## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
warnings.filterwarnings("ignore")


from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

import pickle


## Importing models and vectorizers

In [6]:
!wget --header="Host: doc-14-4k-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Cookie: AUTH_mm5pe75pf4mhpbdikb2k80b74s62g311=08255761277798433337|1644908025000|1vac2lt4gnquk0nmpg1ki1drj7ihkrlb" --header="Connection: keep-alive" "https://doc-14-4k-docs.googleusercontent.com/docs/securesc/lo66ka9s3k9ovif7kku17kum7962ok4j/2q26j9euag6l3gho3krv49ecrvk06bm8/1644908175000/08255761277798433337/08255761277798433337/1mw_EPvksRBgOQO2ELw8ZofOXQ5FBzOoD?e=download&authuser=0" -c -O 'sms_spam_detection_vectorisers.zip'

--2022-02-15 06:57:42--  https://doc-14-4k-docs.googleusercontent.com/docs/securesc/lo66ka9s3k9ovif7kku17kum7962ok4j/2q26j9euag6l3gho3krv49ecrvk06bm8/1644908175000/08255761277798433337/08255761277798433337/1mw_EPvksRBgOQO2ELw8ZofOXQ5FBzOoD?e=download&authuser=0
Resolving doc-14-4k-docs.googleusercontent.com (doc-14-4k-docs.googleusercontent.com)... 108.177.11.132, 2607:f8b0:400c:c01::84
Connecting to doc-14-4k-docs.googleusercontent.com (doc-14-4k-docs.googleusercontent.com)|108.177.11.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1716637 (1.6M) [application/zip]
Saving to: ‘sms_spam_detection_vectorisers.zip’

sms_spam_detection_ 100%[===================>]   1.64M  --.-KB/s    in 0.01s   

2022-02-15 06:57:43 (124 MB/s) - ‘sms_spam_detection_vectorisers.zip’ saved [1716637/1716637]



In [7]:
!unzip sms_spam_detection_vectorisers.zip

Archive:  sms_spam_detection_vectorisers.zip
  inflating: best_rf_model.pkl       
  inflating: __MACOSX/._best_rf_model.pkl  
  inflating: scaler.pkl              
  inflating: __MACOSX/._scaler.pkl   
  inflating: tf_idf_vectorizer.pkl   
  inflating: __MACOSX/._tf_idf_vectorizer.pkl  


## Pre-process data

In [16]:
# Ref: https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase



import string
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuation_free="".join([i for i in text if i not in string.punctuation])
    return punctuation_free





#defining function for tokenization
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

 
def tokenization(text):
    # tokens = re.split('W+',text)
    tokens = word_tokenize(text)
    return tokens



#importing nlp library
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')


#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    
    output= [i for i in text if i not in stopwords]
    # print(output)
    return output






from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text







def preprocess_data(sms_text):

  sms_spam_data = pd.DataFrame()
  sms_spam_data['original_text'] = [sms_text]
  sms_spam_data['clean_msg']= sms_spam_data['original_text'].apply(lambda x: decontractions(x))
  sms_spam_data['clean_msg']= sms_spam_data['clean_msg'].apply(lambda x:remove_punctuation(x))
  sms_spam_data['clean_msg']= sms_spam_data['clean_msg'].apply(lambda x: x.lower())
  sms_spam_data['clean_msg']= sms_spam_data['clean_msg'].apply(lambda x: tokenization(x))
  sms_spam_data['clean_msg']= sms_spam_data['clean_msg'].apply(lambda x:remove_stopwords(x))
  sms_spam_data['clean_msg']=sms_spam_data['clean_msg'].apply(lambda x:lemmatizer(x))
  sms_spam_data['clean_msg_length'] = sms_spam_data["clean_msg"].str.len()

  return sms_spam_data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Featurisation

In [17]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer



In [18]:
def featurize(sms_spam_data):
  sms_spam_data["clean_msg"] = sms_spam_data["clean_msg"].map(' '.join)
  sms_spam_data = featurize_clean_msg(sms_spam_data)
  # print(sms_spam_data.shape)
  sms_spam_data = featurize_clean_msg_length(sms_spam_data)
  # print(sms_spam_data.shape)
  sms_spam_data = sms_spam_data.drop(['original_text', 'clean_msg', 'clean_msg_length'], axis = 1)
  
  # print(sms_spam_data.shape)
  return sms_spam_data

  


### Applying tfidf vectorizer to sms text feature

In [19]:
# Ref - https://stackoverflow.com/questions/43601358/empty-vocabulary-for-single-letter-by-countvectorizer
def featurize_clean_msg(sms_spam_data):

  tf_idf = pickle.load(open("tf_idf_vectorizer.pkl", 'rb'))
  # tf_idf = TfidfVectorizer(token_pattern = r"(?u)\b\w+\b")
  sms_tf_idf = tf_idf.transform(sms_spam_data['clean_msg'])
  clean_msg_features = tf_idf.get_feature_names()
  print(sms_spam_data.columns)
  print(sms_tf_idf.shape)
  sms_spam_data[clean_msg_features] = pd.DataFrame(sms_tf_idf.todense(), index=sms_spam_data.index)
  print("In featurize_clean_msg after")
  print(sms_spam_data.shape)
  return sms_spam_data

### Applying min max scaler on sms length

In [20]:
from sklearn.preprocessing import MinMaxScaler

def featurize_clean_msg_length(sms_spam_data):
    scaler = pickle.load(open("scaler.pkl", 'rb'))
    # scaler = MinMaxScaler()
    # scaler.fit(sms_spam_data['clean_msg_length'].values.reshape(-1, 1))
    sms_spam_data['nrm_clean_msg_length']=scaler.transform(sms_spam_data['clean_msg_length'].values.reshape(-1, 1))

    return sms_spam_data


## Predict function

In [21]:
def predict_spam(sms_spam_data):
  model = pickle.load(open("best_rf_model.pkl", 'rb'))
  return model.predict(sms_spam_data)

## Final function

In [27]:
import time

def final(sms_text):
  
  start = time.time()
  
  
  # pre-process data
  sms_spam_data = preprocess_data(sms_text)

  # featurize data
  sms_spam_data = featurize(sms_spam_data)
  # predict
  spam = predict_spam(sms_spam_data)

  end = time.time()
  print("Took {0} seconds to predict".format(end - start))

  print("Spam or not?", spam[0])

  return spam[0]


In [28]:
final("hurray! you won $20 reward!")

Index(['original_text', 'clean_msg', 'clean_msg_length'], dtype='object')
(1, 7849)
In featurize_clean_msg after
(1, 7852)
Took 7.252431154251099 seconds to predict
Spam or not? 0


0

### Deploying web app using streamlit

In [29]:
!pip install streamlit


     |████████████████████████████████| 9.7 MB 5.1 MB/s 
     |████████████████████████████████| 76 kB 4.5 MB/s 
     |████████████████████████████████| 111 kB 58.2 MB/s 
     |████████████████████████████████| 180 kB 61.5 MB/s 
     |████████████████████████████████| 4.3 MB 48.5 MB/s 
     |████████████████████████████████| 164 kB 55.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 128 kB 46.7 MB/s 
     |████████████████████████████████| 792 kB 41.4 MB/s 
     |████████████████████████████████| 380 kB 57.0 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=e2ccb121113c72c4a148310afb69af3ac36e811051c694857e320f5f9bf6014f
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

In [32]:
%%writefile sms_app.py
import streamlit as st
import time
import os
import pandas as pd
import numpy as np
import pickle
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler





def final(sms_text):
  
  start = time.time()
  
  
  # pre-process data
  sms_spam_data = preprocess_data(sms_text)

  # featurize data
  sms_spam_data = featurize(sms_spam_data)
  # predict
  spam = predict_spam(sms_spam_data)

  end = time.time()
  print("Took {0} seconds to predict".format(end - start))

  print("Spam or not?", spam[0])

  return spam[0]






  # Ref: https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase



#defining the function to remove punctuation
def remove_punctuation(text):
    punctuation_free="".join([i for i in text if i not in string.punctuation])
    return punctuation_free





#defining function for tokenization
nltk.download('punkt')

 
def tokenization(text):
    # tokens = re.split('W+',text)
    tokens = word_tokenize(text)
    return tokens



#importing nlp library
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')


#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    
    output= [i for i in text if i not in stopwords]
    # print(output)
    return output






nltk.download('wordnet')

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text







def preprocess_data(sms_text):

  sms_spam_data = pd.DataFrame()
  sms_spam_data['original_text'] = [sms_text]
  sms_spam_data['clean_msg']= sms_spam_data['original_text'].apply(lambda x: decontractions(x))
  sms_spam_data['clean_msg']= sms_spam_data['clean_msg'].apply(lambda x:remove_punctuation(x))
  sms_spam_data['clean_msg']= sms_spam_data['clean_msg'].apply(lambda x: x.lower())
  sms_spam_data['clean_msg']= sms_spam_data['clean_msg'].apply(lambda x: tokenization(x))
  sms_spam_data['clean_msg']= sms_spam_data['clean_msg'].apply(lambda x:remove_stopwords(x))
  sms_spam_data['clean_msg']=sms_spam_data['clean_msg'].apply(lambda x:lemmatizer(x))
  sms_spam_data['clean_msg_length'] = sms_spam_data["clean_msg"].str.len()

  return sms_spam_data



def featurize(sms_spam_data):
  sms_spam_data["clean_msg"] = sms_spam_data["clean_msg"].map(' '.join)
  sms_spam_data = featurize_clean_msg(sms_spam_data)
  # print(sms_spam_data.shape)
  sms_spam_data = featurize_clean_msg_length(sms_spam_data)
  # print(sms_spam_data.shape)
  sms_spam_data = sms_spam_data.drop(['original_text', 'clean_msg', 'clean_msg_length'], axis = 1)
  
  # print(sms_spam_data.shape)
  return sms_spam_data

  

# Ref - https://stackoverflow.com/questions/43601358/empty-vocabulary-for-single-letter-by-countvectorizer
def featurize_clean_msg(sms_spam_data):

  tf_idf = pickle.load(open("tf_idf_vectorizer.pkl", 'rb'))
  # tf_idf = TfidfVectorizer(token_pattern = r"(?u)\b\w+\b")
  sms_tf_idf = tf_idf.transform(sms_spam_data['clean_msg'])
  clean_msg_features = tf_idf.get_feature_names()
  #print(sms_spam_data.columns)
  #print(sms_tf_idf.shape)
  sms_spam_data[clean_msg_features] = pd.DataFrame(sms_tf_idf.todense(), index=sms_spam_data.index)
  #print("In featurize_clean_msg after")
  #print(sms_spam_data.shape)
  return sms_spam_data





def featurize_clean_msg_length(sms_spam_data):
    scaler = pickle.load(open("scaler.pkl", 'rb'))
    # scaler = MinMaxScaler()
    # scaler.fit(sms_spam_data['clean_msg_length'].values.reshape(-1, 1))
    sms_spam_data['nrm_clean_msg_length']=scaler.transform(sms_spam_data['clean_msg_length'].values.reshape(-1, 1))

    return sms_spam_data




def predict_spam(sms_spam_data):
  model = pickle.load(open("best_rf_model.pkl", 'rb'))
  return model.predict(sms_spam_data)





def main():
  st.title("SMS Spam Detection")	
  text = st.text_input("Enter Your Text", "")
 
# display the name when the submit button is clicked
# .title() is used to get the input text string
  if(st.button('Submit')):
    sms_text = text.title()
    spam_detection = final(sms_text)
    if(spam_detection == 0):
      st.success("Not a spam message")
    else:
      st.error("It is a spam message")  
 
  
if __name__ == '__main__':
  main()


Overwriting sms_app.py


In [33]:
!streamlit run sms_app.py --server.enableWebsocketCompression=false & npx localtunnel --port 8501 

2022-02-15 07:23:57.555 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.897s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.243.210.78:8501

your url is: https://moody-lion-97.loca.lt
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloadi